In [1]:
# default_exp values
# hide
_FNAME='values'

import unittest
from unittest import mock
from nbdev.export import notebook2script
import os

TESTCASE = unittest.TestCase()
_nbpath = os.path.join(_dh[0], _FNAME+'.ipynb')

In [2]:
#export
import random
import json
import sunspec.core.suns as suns
from shadowsunspec.factory import assemble, encode, ShadowSunspecEncoder
import shadowsunspec.defaults as defaults

In [3]:
#export

class ExampleValue:
    def __init__(self, values, mid, symbol, block_type=suns.SUNS_BLOCK_FIXED):
        try:
            self.values = list(values)
        except TypeError:
            self.values = [values]
            
        self.mid = mid
        self.symbol = symbol
        self.block_type = block_type
        
    def generate(self):
        return random.choice(self.values)
    
    def attach(self, dev):
        mid = self.mid
        symbol = self.symbol
        block_type = self.block_type
        
        models = dev.models.get(mid) or []
        altered = []
        for model in models:
            blocks = [b for b in model.blocks if b.block_type.type==block_type]
            for block in blocks:
                point = block.points[symbol]
                point.example = self
                altered.append(point)
        return altered

class ExampleValues:
    def __init__(self):
        self.examples = {}
    
    def key(self, mid, symbol):
        return '{mid}.{symbol}'.format(mid=mid, symbol=symbol)
    def add(self, *args, **kwargs):
        newexample = ExampleValue(*args, **kwargs)
        key = self.key(newexample.mid, newexample.symbol)
        self.examples[key] = newexample
    def attach(self, dev):
        for example in self.examples.values():
            example.attach(dev)

In [4]:
def test_example_attachment():
    dev = assemble([{'mid': 802}, {'mid': 805, 'repeating': 4}])
    ev = ExampleValue(3, mid=802, symbol='V')
    altered = ev.attach(dev)
    v = dev.models[802][0].blocks[0].points['V']
    TESTCASE.assertIs(altered[0], v)
    TESTCASE.assertIsInstance(v.example, ExampleValue)
    TESTCASE.assertEqual(v.example.generate(), 3)
    print("SUCCESS!")
    
test_example_attachment()    

SUCCESS!


In [5]:
def test_multi_attachment():    
    dev = assemble([{'mid': 802}, {'mid': 805, 'repeating': 4}])
    ev2 = ExampleValue(2, mid=805, symbol='CellTmp', block_type='repeating')
    altered = ev2.attach(dev)
    TESTCASE.assertEqual(len(altered), 4)
    print("SUCCESS!")
test_multi_attachment()

SUCCESS!


In [6]:
#export
def get_point_value(point):
    value = point.value_base
    if value is None:
        try:
            value = point.example.generate()
        except AttributeError:  #no example is given
            value = defaults.unimplemented[point.point_type.type]
    return value

In [7]:
import sunspec.core.device as device

def test_get_point_value_unimpl():
    int16 = device.PointType(pid='Test', ptype=suns.SUNS_TYPE_INT16)
    p1 = device.Point(point_type=int16)
    TESTCASE.assertIsNone(p1.value_base)
    TESTCASE.assertEqual(get_point_value(p1), suns.SUNS_UNIMPL_INT16)
    p1.value_base = 123
    TESTCASE.assertEqual(get_point_value(p1), 123)
    print("SUCCESS!")
test_get_point_value_unimpl()

SUCCESS!


In [8]:
def test_get_point_value():
    dev = assemble([{'mid': 802}, {'mid': 805, 'repeating': 4}])
    ev2 = ExampleValue(2, mid=805, symbol='CellTmp', block_type='repeating')
    altered = ev2.attach(dev)
    pts = dev.models[805][0].blocks[1].points_list

    TESTCASE.assertEqual([get_point_value(pt) for pt in pts],
                        [suns.SUNS_UNIMPL_UINT16, 2, suns.SUNS_UNIMPL_BITFIELD32])
    print("SUCCESS!")
    

test_get_point_value()

SUCCESS!


## Example Values

In [9]:
#export
class ShadowSunspecValueEncoder(ShadowSunspecEncoder):
    def encode_point(self, point):
        pt = point.point_type
        value = get_point_value(point)
        return {pt.id: value}

In [10]:
#export
example_device = assemble([{'mid': 802}, {'mid': 805, 'repeating': 4}])
ev = ExampleValues()
ev.add(1, mid=805, symbol='StrIdx')
ev.add(16, mid=805, symbol='NCell')
ev.add(range(470, 520), mid=805, symbol='V')

ev.add(range(280, 330), mid=805, symbol='CellV', block_type='repeating')
ev.add(0, mid=805, symbol='CellSt', block_type='repeating')
ev.add(range(300, 340), mid=805, symbol='CellTmp', block_type='repeating')

ev.add(range(470, 520), mid=802, symbol='V')
ev.add(range(0, 1000), mid=802, symbol='A')
ev.add(range(0, 600), mid=802, symbol='W')  #assuming W_SF=1
ev.add(range(470, 520), mid=802, symbol='SoC')

example_values=ev

ev.attach(example_device)

In [11]:
notebook2script(_nbpath)

Converted values.ipynb.
